<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#데이터-준비와-전처리" data-toc-modified-id="데이터-준비와-전처리-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>데이터 준비와 전처리</a></span><ul class="toc-item"><li><span><a href="#인덱싱" data-toc-modified-id="인덱싱-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>인덱싱</a></span></li></ul></li><li><span><a href="#분석하기" data-toc-modified-id="분석하기-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>분석하기</a></span></li><li><span><a href="#내가-선호하는-영화를-5가지-골라서-rating에-추가해주기" data-toc-modified-id="내가-선호하는-영화를-5가지-골라서-rating에-추가해주기-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>내가 선호하는 영화를 5가지 골라서 rating에 추가해주기</a></span></li><li><span><a href="#CSR-matrix를-직접-만들어-보기" data-toc-modified-id="CSR-matrix를-직접-만들어-보기-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>CSR matrix를 직접 만들어 보기</a></span></li><li><span><a href="#als_model-=-AlternatingLeastSquares-모델을-직접-구성하여-훈련시켜-보기" data-toc-modified-id="als_model-=-AlternatingLeastSquares-모델을-직접-구성하여-훈련시켜-보기-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 보기</a></span></li><li><span><a href="#내가-선호하는-5가지-영화-중-하나와-그-외의-영화-하나를-골라-훈련된-모델이-예측한-나의-선호도를-파악하기" data-toc-modified-id="내가-선호하는-5가지-영화-중-하나와-그-외의-영화-하나를-골라-훈련된-모델이-예측한-나의-선호도를-파악하기-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악하기</a></span></li><li><span><a href="#내가-좋아하는-영화와-비슷한-영화를-추천받아-보기" data-toc-modified-id="내가-좋아하는-영화와-비슷한-영화를-추천받아-보기-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>내가 좋아하는 영화와 비슷한 영화를 추천받아 보기</a></span></li><li><span><a href="#내가-가장-좋아할-만한-영화들을-추천받아-보기" data-toc-modified-id="내가-가장-좋아할-만한-영화들을-추천받아-보기-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>내가 가장 좋아할 만한 영화들을 추천받아 보기</a></span></li><li><span><a href="#마치며" data-toc-modified-id="마치며-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>마치며</a></span></li></ul></div>

## 데이터 준비와 전처리

In [1]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook

In [2]:
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
original_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [3]:
# 3점 이상만 남기기
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'original_data_size: {original_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / original_data_size:.2%}')

original_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [4]:
# rating 컬럼의 이름을 count로 바꾸기
ratings.rename(columns={'rating':'count'}, inplace=True)
ratings.head()

,user_id,movie_id,count,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [5]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 836478 entries, 0 to 1000208
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype
---  ------     --------------   -----
 0   user_id    836478 non-null  int64
 1   movie_id   836478 non-null  int64
 2   count      836478 non-null  int64
 3   timestamp  836478 non-null  int64
dtypes: int64(4)
memory usage: 31.9 MB


In [6]:
# 영화 제목을 보기 위해서 메타 데이터 읽어오기
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre']
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
movies[movies['movie_id']==661]

,movie_id,title,genre
655,661,James and the Giant Peach (1996),Animation|Children's|Musical


In [8]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3883 entries, 0 to 3882
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   movie_id  3883 non-null   int64 
 1   title     3883 non-null   object
 2   genre     3883 non-null   object
dtypes: int64(1), object(2)
memory usage: 91.1+ KB


### 인덱싱

In [9]:
# 유저, 영화로 배열 만들어주기
user_unique = ratings['user_id'].unique()
user_unique

array([   1,    2,    3, ..., 6038, 6039, 6040])

In [10]:
movie_unique = ratings['movie_id'].unique()
movie_unique

array([1193,  661,  914, ...,  690, 2909, 1360])

In [11]:
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [12]:
# 개수 확인하기
print(movies['movie_id'].nunique())
print(movies['title'].nunique())
print(movies['genre'].nunique())

3883
3883
301


In [13]:
# 배열 만들기
movie_unique = movies['movie_id'].unique()
print(movie_unique)
title_unique = movies['title'].unique()
print(title_unique)

[   1    2    3 ... 3950 3951 3952]
['Toy Story (1995)' 'Jumanji (1995)' 'Grumpier Old Men (1995)' ...
 'Tigerland (2000)' 'Two Family House (2000)' 'Contender, The (2000)']


In [14]:
# movie_id와 title로 인덱싱 해주기
id_to_title = {i:j for i, j in zip(movie_unique, title_unique)}
id_to_title

{1: 'Toy Story (1995)',
 2: 'Jumanji (1995)',
 3: 'Grumpier Old Men (1995)',
 4: 'Waiting to Exhale (1995)',
 5: 'Father of the Bride Part II (1995)',
 6: 'Heat (1995)',
 7: 'Sabrina (1995)',
 8: 'Tom and Huck (1995)',
 9: 'Sudden Death (1995)',
 10: 'GoldenEye (1995)',
 11: 'American President, The (1995)',
 12: 'Dracula: Dead and Loving It (1995)',
 13: 'Balto (1995)',
 14: 'Nixon (1995)',
 15: 'Cutthroat Island (1995)',
 16: 'Casino (1995)',
 17: 'Sense and Sensibility (1995)',
 18: 'Four Rooms (1995)',
 19: 'Ace Ventura: When Nature Calls (1995)',
 20: 'Money Train (1995)',
 21: 'Get Shorty (1995)',
 22: 'Copycat (1995)',
 23: 'Assassins (1995)',
 24: 'Powder (1995)',
 25: 'Leaving Las Vegas (1995)',
 26: 'Othello (1995)',
 27: 'Now and Then (1995)',
 28: 'Persuasion (1995)',
 29: 'City of Lost Children, The (1995)',
 30: 'Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)',
 31: 'Dangerous Minds (1995)',
 32: 'Twelve Monkeys (1995)',
 33: 'Wings of Courage (1995)',
 34: 'Babe (1

In [15]:
# movie_id와 genre 인덱싱
df = movies[['movie_id', 'genre']]
id_to_genre = {k:v for k, v in zip(df.iloc[:, 0], df.iloc[:, 1])}
id_to_genre

{1: "Animation|Children's|Comedy",
 2: "Adventure|Children's|Fantasy",
 3: 'Comedy|Romance',
 4: 'Comedy|Drama',
 5: 'Comedy',
 6: 'Action|Crime|Thriller',
 7: 'Comedy|Romance',
 8: "Adventure|Children's",
 9: 'Action',
 10: 'Action|Adventure|Thriller',
 11: 'Comedy|Drama|Romance',
 12: 'Comedy|Horror',
 13: "Animation|Children's",
 14: 'Drama',
 15: 'Action|Adventure|Romance',
 16: 'Drama|Thriller',
 17: 'Drama|Romance',
 18: 'Thriller',
 19: 'Comedy',
 20: 'Action',
 21: 'Action|Comedy|Drama',
 22: 'Crime|Drama|Thriller',
 23: 'Thriller',
 24: 'Drama|Sci-Fi',
 25: 'Drama|Romance',
 26: 'Drama',
 27: 'Drama',
 28: 'Romance',
 29: 'Adventure|Sci-Fi',
 30: 'Drama',
 31: 'Drama',
 32: 'Drama|Sci-Fi',
 33: 'Adventure|Romance',
 34: "Children's|Comedy|Drama",
 35: 'Drama|Romance',
 36: 'Drama',
 37: 'Documentary',
 38: 'Comedy',
 39: 'Comedy|Romance',
 40: 'Drama',
 41: 'Drama|War',
 42: 'Action|Crime|Drama',
 43: 'Drama',
 44: 'Action|Adventure',
 45: 'Comedy|Drama',
 46: 'Drama|Romance',

In [16]:
# 복사
movie_rating = ratings.copy()
movie_rating

,user_id,movie_id,count,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000203,6040,1090,3,956715518
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [17]:
# title, genre 칼럼 추가
movie_rating['title'] = np.nan
movie_rating['genre'] = np.nan
movie_rating

,user_id,movie_id,count,timestamp,title,genre
0,1,1193,5,978300760,NaN,NaN
1,1,661,3,978302109,NaN,NaN
2,1,914,3,978301968,NaN,NaN
3,1,3408,4,978300275,NaN,NaN
4,1,2355,5,978824291,NaN,NaN
...,...,...,...,...,...,...
1000203,6040,1090,3,956715518,NaN,NaN
1000205,6040,1094,5,956704887,NaN,NaN
1000206,6040,562,5,956704746,NaN,NaN
1000207,6040,1096,4,956715648,NaN,NaN


In [18]:
# title, genre 컬럼에 movie_id 넣어주기
movie_rating['title'] = movie_rating['movie_id']
movie_rating['genre'] = movie_rating['movie_id']
movie_rating

,user_id,movie_id,count,timestamp,title,genre
0,1,1193,5,978300760,1193,1193
1,1,661,3,978302109,661,661
2,1,914,3,978301968,914,914
3,1,3408,4,978300275,3408,3408
4,1,2355,5,978824291,2355,2355
...,...,...,...,...,...,...
1000203,6040,1090,3,956715518,1090,1090
1000205,6040,1094,5,956704887,1094,1094
1000206,6040,562,5,956704746,562,562
1000207,6040,1096,4,956715648,1096,1096


In [19]:
# map 함수를 통해서 바꿔주기
temp_title = movie_rating['title'].map(id_to_title.get).dropna()
temp_title

0          One Flew Over the Cuckoo's Nest (1975)
1                James and the Giant Peach (1996)
2                             My Fair Lady (1964)
3                          Erin Brockovich (2000)
4                            Bug's Life, A (1998)
                            ...                  
1000203                            Platoon (1986)
1000205                   Crying Game, The (1992)
1000206           Welcome to the Dollhouse (1995)
1000207                    Sophie's Choice (1982)
1000208         E.T. the Extra-Terrestrial (1982)
Name: title, Length: 836478, dtype: object

In [20]:
if len(temp_title) == len(movie_rating):
    movie_rating['title'] = temp_title
    print('성공!')
else:
    print('실패!')

movie_rating

성공!


,user_id,movie_id,count,timestamp,title,genre
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),1193
1,1,661,3,978302109,James and the Giant Peach (1996),661
2,1,914,3,978301968,My Fair Lady (1964),914
3,1,3408,4,978300275,Erin Brockovich (2000),3408
4,1,2355,5,978824291,"Bug's Life, A (1998)",2355
...,...,...,...,...,...,...
1000203,6040,1090,3,956715518,Platoon (1986),1090
1000205,6040,1094,5,956704887,"Crying Game, The (1992)",1094
1000206,6040,562,5,956704746,Welcome to the Dollhouse (1995),562
1000207,6040,1096,4,956715648,Sophie's Choice (1982),1096


In [21]:
# 같은 방법으로 genre 컬럼도 변경
temp_genre = movie_rating['genre'].map(id_to_genre.get).dropna()

if len(temp_genre) == len(movie_rating):
    movie_rating['genre'] = temp_genre
    print('성공!')
else:
    print('실패!')

movie_rating.head()

성공!


,user_id,movie_id,count,timestamp,title,genre
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical
2,1,914,3,978301968,My Fair Lady (1964),Musical|Romance
3,1,3408,4,978300275,Erin Brockovich (2000),Drama
4,1,2355,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy


In [22]:
cols = ['user_id', 'movie_id', 'title', 'genre', 'count', 'timestamp']
movie_rating = movie_rating[cols]
movie_rating.head()

,user_id,movie_id,title,genre,count,timestamp
0,1,1193,One Flew Over the Cuckoo's Nest (1975),Drama,5,978300760
1,1,661,James and the Giant Peach (1996),Animation|Children's|Musical,3,978302109
2,1,914,My Fair Lady (1964),Musical|Romance,3,978301968
3,1,3408,Erin Brockovich (2000),Drama,4,978300275
4,1,2355,"Bug's Life, A (1998)",Animation|Children's|Comedy,5,978824291


## 분석하기

+ ratings에 있는 유니크한 영화 개수
+ ratings에 있는 유니크한 사용자 수
+ 가장 인기 있는 영화 30개 (인기순)

In [23]:
movie_rating.head()

,user_id,movie_id,title,genre,count,timestamp
0,1,1193,One Flew Over the Cuckoo's Nest (1975),Drama,5,978300760
1,1,661,James and the Giant Peach (1996),Animation|Children's|Musical,3,978302109
2,1,914,My Fair Lady (1964),Musical|Romance,3,978301968
3,1,3408,Erin Brockovich (2000),Drama,4,978300275
4,1,2355,"Bug's Life, A (1998)",Animation|Children's|Comedy,5,978824291


In [24]:
# rating에 있는 영화 개수
ratings['movie_id'].nunique()

3628

In [25]:
# ratings에 있는 사용자 수
ratings['user_id'].nunique()

6039

In [26]:
# 가장 인기 있는 영화 30개
movie_count = ratings.groupby('movie_id')['user_id'].count()
movie_count.sort_values(ascending=False).head(30)

movie_id
2858    3211
260     2910
1196    2885
1210    2716
2028    2561
589     2509
593     2498
1198    2473
1270    2460
2571    2434
480     2413
2762    2385
608     2371
110     2314
1580    2297
527     2257
1197    2252
2396    2213
1617    2210
318     2194
858     2167
1265    2121
1097    2102
2997    2066
2716    2051
296     2030
356     2022
1240    2019
1       2000
457     1941
Name: user_id, dtype: int64

In [27]:
movie_count = movie_rating.groupby(['movie_id', 'title'])['user_id'].count()
movie_count.sort_values(ascending=False).head(60)

movie_id  title                                                
2858      American Beauty (1999)                                   3211
260       Star Wars: Episode IV - A New Hope (1977)                2910
1196      Star Wars: Episode V - The Empire Strikes Back (1980)    2885
1210      Star Wars: Episode VI - Return of the Jedi (1983)        2716
2028      Saving Private Ryan (1998)                               2561
589       Terminator 2: Judgment Day (1991)                        2509
593       Silence of the Lambs, The (1991)                         2498
1198      Raiders of the Lost Ark (1981)                           2473
1270      Back to the Future (1985)                                2460
2571      Matrix, The (1999)                                       2434
480       Jurassic Park (1993)                                     2413
2762      Sixth Sense, The (1999)                                  2385
608       Fargo (1996)                                             2371


In [28]:
# 어떤 장르가 인기가 많은지 알아보기
genre_count = movie_rating.groupby('genre')['user_id'].count()
genre_count.sort_values(ascending=False).head(30)

genre
Drama                               99388
Comedy                              94264
Comedy|Drama                        36871
Comedy|Romance                      35888
Drama|Romance                       24835
Action|Thriller                     22675
Drama|Thriller                      16133
Horror                              15260
Thriller                            14925
Action|Adventure|Sci-Fi             14277
Drama|War                           13766
Action|Sci-Fi|Thriller              11657
Action|Drama|War                    11316
Crime|Drama                         10960
Action|Sci-Fi                       10594
Action                               9930
Comedy|Drama|Romance                 9804
Action|Adventure                     8744
Action|Drama                         8611
Comedy|Sci-Fi                        7797
Comedy|Horror                        7523
Animation|Children's                 7461
Animation|Children's|Musical         7237
Animation|Children's|Comedy 

In [29]:
# 유저별 몇 개의 영화를 보는지에 대한 통계
user_count = ratings.groupby('user_id')['movie_id'].count()
user_count

user_id
1        53
2       116
3        46
4        19
5       143
       ... 
6036    708
6037    189
6038     18
6039    119
6040    276
Name: movie_id, Length: 6039, dtype: int64

In [30]:
user_count.describe()

count    6039.000000
mean      138.512668
std       156.241599
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: movie_id, dtype: float64

In [31]:
movie_rating.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 836478 entries, 0 to 1000208
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   user_id    836478 non-null  int64 
 1   movie_id   836478 non-null  int64 
 2   title      836478 non-null  object
 3   genre      836478 non-null  object
 4   count      836478 non-null  int64 
 5   timestamp  836478 non-null  int64 
dtypes: int64(4), object(2)
memory usage: 44.7+ MB


## 내가 선호하는 영화를 5가지 골라서 rating에 추가해주기

In [32]:
df = ratings.iloc[:,:3]
df.rename(columns={'count':'rating'}, inplace=True)
df.head()

/home/ssac28/anaconda3/envs/aiffel/lib/python3.7/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,user_id,movie_id,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5


In [33]:
# 내가 좋아하는 영화 5개
my_favorite = [1580, 356, 480, 1036, 1291]

# 'mingki'로 user_id 추가
my_list = pd.DataFrame({'user_id':[8888]*5, 'movie_id':my_favorite, 'rating':[5.0]*5})

if not df.isin({'user_id':[8888]})['user_id'].any():
    df = df.append(my_list)

df.tail(10)

,user_id,movie_id,rating
1000203,6040,1090,3.0
1000205,6040,1094,5.0
1000206,6040,562,5.0
1000207,6040,1096,4.0
1000208,6040,1097,4.0
0,8888,1580,5.0
1,8888,356,5.0
2,8888,480,5.0
3,8888,1036,5.0
4,8888,1291,5.0


In [34]:
df

,user_id,movie_id,rating
0,1,1193,5.0
1,1,661,3.0
2,1,914,3.0
3,1,3408,4.0
4,1,2355,5.0
...,...,...,...
0,8888,1580,5.0
1,8888,356,5.0
2,8888,480,5.0
3,8888,1036,5.0


In [35]:
6040 * 3628

21913120

In [36]:
358869 * 291347

104555406543

##  CSR matrix를 직접 만들어 보기

In [37]:
df.rating

0    5.0
1    3.0
2    3.0
3    4.0
4    5.0
    ... 
0    5.0
1    5.0
2    5.0
3    5.0
4    5.0
Name: rating, Length: 836483, dtype: float64

In [38]:
# CSR Matrix 만들어보기
from scipy.sparse import csr_matrix

num_user = df['user_id'].nunique()
num_movie = df['movie_id'].nunique()
print(num_user, num_movie, '\n')

csr_data = csr_matrix((df.rating, (df.user_id, df.movie_id)))
print(csr_data)

6040 3628 

  (1, 1)	5.0
  (1, 48)	5.0
  (1, 150)	5.0
  (1, 260)	4.0
  (1, 527)	5.0
  (1, 531)	4.0
  (1, 588)	4.0
  (1, 594)	4.0
  (1, 595)	5.0
  (1, 608)	4.0
  (1, 661)	3.0
  (1, 720)	3.0
  (1, 745)	3.0
  (1, 783)	4.0
  (1, 914)	3.0
  (1, 919)	4.0
  (1, 938)	4.0
  (1, 1022)	5.0
  (1, 1028)	5.0
  (1, 1029)	5.0
  (1, 1035)	5.0
  (1, 1097)	4.0
  (1, 1193)	5.0
  (1, 1197)	3.0
  (1, 1207)	4.0
  :	:
  (6040, 3334)	5.0
  (6040, 3342)	3.0
  (6040, 3359)	4.0
  (6040, 3362)	4.0
  (6040, 3418)	3.0
  (6040, 3421)	3.0
  (6040, 3422)	3.0
  (6040, 3449)	3.0
  (6040, 3471)	4.0
  (6040, 3504)	4.0
  (6040, 3505)	4.0
  (6040, 3521)	5.0
  (6040, 3543)	4.0
  (6040, 3547)	4.0
  (6040, 3671)	4.0
  (6040, 3683)	4.0
  (6040, 3703)	4.0
  (6040, 3735)	4.0
  (6040, 3751)	4.0
  (6040, 3819)	5.0
  (8888, 356)	5.0
  (8888, 480)	5.0
  (8888, 1036)	5.0
  (8888, 1291)	5.0
  (8888, 1580)	5.0


## als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 보기

In [39]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [40]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [41]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3953x8889 sparse matrix of type '<class 'numpy.float64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [42]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

## 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악하기

In [43]:
mingki_vector, forrestgump_vector = als_model.user_factors[8888], als_model.item_factors[356]

In [44]:
mingki_vector

array([ 0.37065783, -0.03345272, -0.00458111,  1.1383675 , -0.46524522,
        0.47041708, -0.4569942 , -0.380851  ,  0.4832071 , -0.718122  ,
        0.49874586,  0.00602309, -0.69065636,  0.37721258, -0.9049285 ,
        0.37992612, -0.3343536 ,  0.3272468 , -0.37698638, -0.36300144,
       -0.18813644,  0.20948054,  0.18477139, -0.49628007, -0.49483985,
       -0.01232571,  0.27562788, -0.15541995,  0.02459669, -0.23707287,
       -0.13181092, -1.0157173 ,  0.39880326, -1.4131066 ,  0.80961645,
        0.36378175,  0.88904655, -0.27379867,  0.5250752 ,  0.1645251 ,
        0.22862995, -0.64513224, -0.5659081 , -0.48709294,  0.57107174,
       -0.5455079 ,  0.12204058,  0.31436384,  0.49580964, -0.08207404,
        0.23958665,  0.3306867 , -0.35140443,  0.04164039,  0.49977747,
        0.36984515,  0.34296936,  0.71977955,  0.2754863 ,  0.09362213,
       -0.53624547, -1.0913013 ,  0.27658328, -0.04387179, -0.45592368,
       -0.2545261 ,  0.8633382 , -0.81415695, -0.29199237, -0.23

In [45]:
forrestgump_vector

array([ 0.02735701,  0.02076961,  0.00267059,  0.04129941, -0.00990398,
        0.00299409, -0.00530819, -0.019352  , -0.02325912, -0.02260712,
        0.01070325, -0.0010095 , -0.01604393, -0.0240071 , -0.02449034,
        0.02678454,  0.00475091,  0.03586138, -0.01193898,  0.00533798,
        0.02894232,  0.01832422,  0.05189501,  0.01681719, -0.03420461,
       -0.03056174,  0.00572578,  0.03351824,  0.02746911, -0.01161645,
        0.02314974, -0.01174224,  0.01387261, -0.01227738,  0.01716176,
        0.0107272 ,  0.03383746,  0.00556716,  0.01106093,  0.02828026,
       -0.01967816, -0.0322133 ,  0.01198172,  0.00886814,  0.01960147,
       -0.00577896, -0.00564067,  0.01049535,  0.03469272,  0.02022273,
       -0.0156625 ,  0.01226945, -0.01761288,  0.02452688,  0.03396979,
        0.03272346,  0.01797841, -0.00812789,  0.01807939,  0.03653699,
        0.01572174,  0.00283822,  0.02904576, -0.01817846, -0.02252039,
        0.01828271,  0.01151297, -0.03137381, -0.00592867,  0.00

In [46]:
# mingki와 forrest gump가 내적하는 코드
np.dot(mingki_vector, forrestgump_vector)

0.6079663

In [47]:
# 다른 영화에 대한 선호도를 어떻게 예측하는지 확인하기
# 스타워즈
starwars = 260
starwars_vector = als_model.item_factors[starwars]
print(np.dot(mingki_vector, starwars_vector))

# 굿윌헌팅
goodwillhunting = 1704
goodwillhunting_vector = als_model.item_factors[goodwillhunting]
print(np.dot(mingki_vector, goodwillhunting_vector))

0.2839272
0.076229066


## 내가 좋아하는 영화와 비슷한 영화를 추천받아 보기

In [48]:
# 포레스트 검프
favorite_movie = 356
similar_movie = als_model.similar_items(favorite_movie, N=15)
similar_movie

[(356, 1.0),
 (1265, 0.5730348),
 (1784, 0.4737288),
 (539, 0.45165074),
 (597, 0.43244815),
 (587, 0.43054715),
 (985, 0.39602616),
 (39, 0.39024672),
 (1393, 0.38333243),
 (1777, 0.37734205),
 (1316, 0.3739293),
 (1408, 0.36922762),
 (357, 0.36734122),
 (2321, 0.3666896),
 (3353, 0.36569732)]

In [49]:
[id_to_title[i[0]] for i in similar_movie]

['Forrest Gump (1994)',
 'Groundhog Day (1993)',
 'As Good As It Gets (1997)',
 'Sleepless in Seattle (1993)',
 'Pretty Woman (1990)',
 'Ghost (1990)',
 'Small Wonders (1996)',
 'Clueless (1995)',
 'Jerry Maguire (1996)',
 'Wedding Singer, The (1998)',
 'Anna (1996)',
 'Last of the Mohicans, The (1992)',
 'Four Weddings and a Funeral (1994)',
 'Pleasantville (1998)',
 'Closer You Get, The (2000)']

In [50]:
# 함수 만들기
def get_similar_movie(movie_id):
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [id_to_title[i[0]] for i in similar_movie]
    return similar_movie
print('굿')

굿


In [51]:
# 맨 인 블랙 : 1580
get_similar_movie(1580)

['Men in Black (1997)',
 'Jurassic Park (1993)',
 'Terminator 2: Judgment Day (1991)',
 'Total Recall (1990)',
 'Independence Day (ID4) (1996)',
 'Fifth Element, The (1997)',
 'Matrix, The (1999)',
 'Schlafes Bruder (Brother of Sleep) (1995)',
 'Face/Off (1997)',
 'Lost World: Jurassic Park, The (1997)']

## 내가 가장 좋아할 만한 영화들을 추천받아 보기

In [52]:
user = 8888 # 내 유저 아이디

movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(1198, 0.4803306),
 (589, 0.4537575),
 (1210, 0.39226618),
 (110, 0.37464428),
 (2916, 0.35299447),
 (457, 0.31105977),
 (2571, 0.3083403),
 (377, 0.2977932),
 (1240, 0.2955552),
 (1196, 0.29035467),
 (260, 0.2839272),
 (1610, 0.2830892),
 (592, 0.27578875),
 (2115, 0.26413542),
 (1270, 0.25676036),
 (1408, 0.23910055),
 (590, 0.21984333),
 (1259, 0.21845275),
 (1127, 0.21637642),
 (2028, 0.2107422)]

In [53]:
[id_to_title[i[0]] for i in movie_recommended]

['Raiders of the Lost Ark (1981)',
 'Terminator 2: Judgment Day (1991)',
 'Star Wars: Episode VI - Return of the Jedi (1983)',
 'Braveheart (1995)',
 'Total Recall (1990)',
 'Fugitive, The (1993)',
 'Matrix, The (1999)',
 'Speed (1994)',
 'Terminator, The (1984)',
 'Star Wars: Episode V - The Empire Strikes Back (1980)',
 'Star Wars: Episode IV - A New Hope (1977)',
 'Hunt for Red October, The (1990)',
 'Batman (1989)',
 'Indiana Jones and the Temple of Doom (1984)',
 'Back to the Future (1985)',
 'Last of the Mohicans, The (1992)',
 'Dances with Wolves (1990)',
 'Stand by Me (1986)',
 'Abyss, The (1989)',
 'Saving Private Ryan (1998)']

In [54]:
# 왜 터미네이터를 추천했을까 확인해보기
# 터미네이터  : 589
explain = als_model.explain(user, csr_data, itemid=589)

In [55]:
explain

(0.44687684978082476,
 [(480, 0.21443864627355808),
  (1580, 0.16233399874639579),
  (1036, 0.09857772951226113),
  (356, -0.011594172076334853),
  (1291, -0.016879352675055397)],
 (array([[ 6.33665887e-01,  1.34250997e-01,  1.33823211e-01, ...,
           1.11876588e-01,  8.74169026e-02,  1.19637916e-01],
         [ 8.50702772e-02,  5.87668676e-01,  1.27140427e-01, ...,
           9.79329169e-02,  5.80720171e-02,  8.05464687e-02],
         [ 8.47992036e-02,  9.26823461e-02,  5.90822312e-01, ...,
           5.89483276e-02,  1.08586398e-01,  7.56840450e-02],
         ...,
         [ 7.08923774e-02,  7.25716511e-02,  6.22509044e-02, ...,
           5.29650795e-01, -6.58283753e-03,  2.21451514e-04],
         [ 5.53931091e-02,  4.58629118e-02,  8.32369784e-02, ...,
           5.51847433e-02,  5.26307866e-01,  1.02678717e-02],
         [ 7.58104661e-02,  6.33961462e-02,  7.09668649e-02, ...,
           7.40811681e-02,  6.29197354e-02,  5.39651082e-01]]),
  False))

+ 0번 인덱스 : 추천 지수
+ 1번 인덱스 : 다른 콘텐츠의 기여도
+ 2번 인덱스 : 벡터

In [56]:
[(id_to_title[i[0]], i[1]) for i in explain[1]]

[('Jurassic Park (1993)', 0.21443864627355808),
 ('Men in Black (1997)', 0.16233399874639579),
 ('Die Hard (1988)', 0.09857772951226113),
 ('Forrest Gump (1994)', -0.011594172076334853),
 ('Indiana Jones and the Last Crusade (1989)', -0.016879352675055397)]

## 마치며

추천 시스템에 대해 전반적으로 이해력이 올라갔다!

새로운 개념들에 대해서도 배웠으며(csr_matrix, AlternatingLeastSquares 모델 등) 우리가 사용하는 넷플릭스나 멜론 등 다양한 곳들에서 어떤 방식으로 추천 시스템을 구성하고 또 작동하는지 깊이 있게는 아니지만 조금은 알게 되었다.

해봤던 것들 중에 가장 흥미가 있었고, 하면서도 내내 즐거웠었다.

아쉬웠던, 조금 더 배우고 싶었던 것을 고르라면 csr_matrix에 대해서 어떤 방식으로 작동하는지 이해하기가 조금은 어려웠다. 어떻게 어떻게 코드를 짜서 실행은 됐지만 이게 왜 실행되는지 잘 이해가 안된 느낌. 다음에 또 기회가 있다면 더 멋진 추천 시스템을 구현해보고 싶다.